# 🧠 Automatic Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Using Frequency-Domain Analysis for Medical Image Segmentation

**What this tool does:**
1. Analyzes medical images using FFT (Fast Fourier Transform)
2. Applies frequency-domain filters to enhance tumor features
3. **Automatically detects if a tumor is present**
4. Highlights the tumor location
5. Estimates tumor size

**How to use:**
1. Run all cells in order
2. Upload your medical image OR use our samples
3. Get automatic tumor detection results!

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

# Download our code from GitHub
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

# Download sample tumor images
!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, disk

# ========== HELPER FUNCTIONS ==========
def load_image_safe(filepath):
    """Load any image and convert to grayscale float [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img).astype(np.float64) / 255.0
    return arr

def load_mask_safe(filepath):
    """Load any image and convert to binary mask"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img)
    return (arr > 127).astype(np.uint8)

# ========== TUMOR DETECTION SYSTEM ==========
def detect_tumor(image, method='combined', sensitivity=0.5):
    """
    Automatic tumor detection using frequency-domain analysis.
    
    Parameters:
    -----------
    image : np.ndarray
        Grayscale image (0-1 range)
    method : str
        'otsu', 'fft_highpass', 'fft_bandpass', or 'combined'
    sensitivity : float
        Detection sensitivity (0-1). Higher = more sensitive (may have more false positives)
    
    Returns:
    --------
    dict with:
        - tumor_detected: bool
        - confidence: float (0-1)
        - tumor_mask: binary mask of detected tumor
        - tumor_location: (row, col) center of tumor
        - tumor_area_percent: percentage of image that is tumor
        - num_regions: number of suspicious regions found
    """
    
    h, w = image.shape
    total_pixels = h * w
    
    # Minimum tumor size (as fraction of image)
    min_tumor_fraction = 0.005 * (1 - sensitivity)  # 0.5% to 0.05% based on sensitivity
    min_tumor_pixels = int(total_pixels * min_tumor_fraction)
    
    # Maximum tumor size (tumors shouldn't be more than 40% of image)
    max_tumor_fraction = 0.4
    max_tumor_pixels = int(total_pixels * max_tumor_fraction)
    
    # Get segmentation masks from different methods
    masks = {}
    
    # Method 1: Otsu thresholding on smoothed image
    smoothed = ndimage.gaussian_filter(image, sigma=2)
    thresh = threshold_otsu(smoothed)
    # Tumors are often brighter OR darker than background
    mask_bright = smoothed > thresh
    mask_dark = smoothed < thresh
    masks['otsu_bright'] = mask_bright
    masks['otsu_dark'] = mask_dark
    
    # Method 2: FFT High-pass (enhances edges/boundaries)
    try:
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=20)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        hp_thresh = threshold_otsu(hp_img)
        masks['fft_hp'] = hp_img > hp_thresh
    except:
        masks['fft_hp'] = np.zeros_like(image, dtype=bool)
    
    # Method 3: FFT Band-pass (tumor-specific frequencies)
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=5, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        bp_thresh = threshold_otsu(bp_img)
        masks['fft_bp'] = bp_img > bp_thresh
    except:
        masks['fft_bp'] = np.zeros_like(image, dtype=bool)
    
    # Choose method
    if method == 'otsu':
        # Use whichever otsu mask has more reasonable tumor size
        bright_area = masks['otsu_bright'].sum()
        dark_area = masks['otsu_dark'].sum()
        if min_tumor_pixels < bright_area < max_tumor_pixels:
            candidate_mask = masks['otsu_bright']
        elif min_tumor_pixels < dark_area < max_tumor_pixels:
            candidate_mask = masks['otsu_dark']
        else:
            candidate_mask = masks['otsu_bright'] if bright_area < dark_area else masks['otsu_dark']
    elif method == 'fft_highpass':
        candidate_mask = masks['fft_hp']
    elif method == 'fft_bandpass':
        candidate_mask = masks['fft_bp']
    else:  # combined
        # Combine methods: look for regions detected by multiple methods
        # Weight FFT methods more heavily
        combined = (masks['otsu_bright'].astype(float) * 0.3 + 
                   masks['fft_hp'].astype(float) * 0.35 + 
                   masks['fft_bp'].astype(float) * 0.35)
        candidate_mask = combined > (0.5 - sensitivity * 0.3)
    
    # Clean up the mask
    candidate_mask = binary_closing(candidate_mask, disk(3))
    candidate_mask = remove_small_objects(candidate_mask, min_size=min_tumor_pixels)
    
    # Label connected regions
    labeled_mask = label(candidate_mask)
    regions = regionprops(labeled_mask, intensity_image=image)
    
    # Filter regions by properties
    valid_regions = []
    for region in regions:
        area = region.area
        # Check size constraints
        if min_tumor_pixels < area < max_tumor_pixels:
            # Check if region is somewhat compact (circularity)
            perimeter = region.perimeter + 1e-8
            circularity = 4 * np.pi * area / (perimeter ** 2)
            if circularity > 0.1:  # Not too elongated
                valid_regions.append(region)
    
    # Create final tumor mask from valid regions
    final_mask = np.zeros_like(image, dtype=bool)
    for region in valid_regions:
        final_mask[labeled_mask == region.label] = True
    
    # Calculate results
    tumor_pixels = final_mask.sum()
    tumor_detected = tumor_pixels > min_tumor_pixels
    tumor_area_percent = (tumor_pixels / total_pixels) * 100
    
    # Calculate confidence based on multiple factors
    if tumor_detected and len(valid_regions) > 0:
        # Factors that increase confidence:
        # 1. Reasonable tumor size (not too small, not too big)
        size_score = min(tumor_area_percent / 5, 1.0) * min(1.0, 20 / (tumor_area_percent + 1))
        
        # 2. Compact shape
        avg_circularity = np.mean([4 * np.pi * r.area / (r.perimeter**2 + 1e-8) for r in valid_regions])
        shape_score = min(avg_circularity * 2, 1.0)
        
        # 3. Contrast with surroundings
        tumor_intensity = image[final_mask].mean() if final_mask.any() else 0
        bg_intensity = image[~final_mask].mean() if (~final_mask).any() else 0
        contrast_score = min(abs(tumor_intensity - bg_intensity) * 3, 1.0)
        
        confidence = (size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4)
        confidence = np.clip(confidence, 0, 1)
    else:
        confidence = 0.0
    
    # Find tumor center
    if tumor_detected and final_mask.any():
        tumor_coords = np.where(final_mask)
        tumor_center = (int(np.mean(tumor_coords[0])), int(np.mean(tumor_coords[1])))
    else:
        tumor_center = None
    
    return {
        'tumor_detected': tumor_detected,
        'confidence': confidence,
        'tumor_mask': final_mask,
        'tumor_location': tumor_center,
        'tumor_area_percent': tumor_area_percent,
        'num_regions': len(valid_regions),
        'all_masks': masks
    }

def display_detection_results(image, detection_result):
    """
    Display tumor detection results with visualization.
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Detection overlay
    axes[1].imshow(image, cmap='gray')
    if detection_result['tumor_detected']:
        axes[1].imshow(detection_result['tumor_mask'], cmap='Reds', alpha=0.6)
        if detection_result['tumor_location']:
            row, col = detection_result['tumor_location']
            axes[1].plot(col, row, 'g+', markersize=20, markeredgewidth=3)
    axes[1].set_title('Detected Tumor Region', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Results summary
    axes[2].axis('off')
    
    if detection_result['tumor_detected']:
        status_color = 'red' if detection_result['confidence'] > 0.5 else 'orange'
        status_text = '⚠️ TUMOR DETECTED' if detection_result['confidence'] > 0.5 else '❓ POSSIBLE TUMOR'
    else:
        status_color = 'green'
        status_text = '✅ NO TUMOR DETECTED'
    
    # Display results text
    results_text = f"""
    
    {status_text}
    
    ─────────────────────────
    
    Confidence: {detection_result['confidence']*100:.1f}%
    
    Tumor Area: {detection_result['tumor_area_percent']:.2f}%
    
    Suspicious Regions: {detection_result['num_regions']}
    
    """
    
    if detection_result['tumor_location']:
        results_text += f"Location: ({detection_result['tumor_location'][1]}, {detection_result['tumor_location'][0]})"
    
    axes[2].text(0.1, 0.5, results_text, fontsize=14, verticalalignment='center',
                fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[2].set_title('Detection Results', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print('\n' + '='*50)
    if detection_result['tumor_detected']:
        if detection_result['confidence'] > 0.7:
            print('🔴 HIGH CONFIDENCE: Tumor likely present')
        elif detection_result['confidence'] > 0.4:
            print('🟡 MEDIUM CONFIDENCE: Possible tumor - recommend further review')
        else:
            print('🟠 LOW CONFIDENCE: Suspicious region detected - needs verification')
    else:
        print('🟢 No significant tumor regions detected')
    print('='*50)

# Initialize
image = None
mask = None
print('✅ Setup complete! Tumor detection system ready.')

---
## 📁 Load Your Image

**Run ONE of the following cells:**

In [ ]:
# ========== OPTION A: UPLOAD YOUR OWN IMAGE ==========
print('Upload your medical image (MRI, CT, etc.):')
print('Supported formats: PNG, JPG, JPEG, BMP\n')

uploaded = files.upload()

if uploaded:
    uploaded_files = list(uploaded.keys())
    image = load_image_safe(uploaded_files[0])
    mask = None  # No ground truth for uploaded images
    
    print(f'\n✅ Loaded: {uploaded_files[0]}')
    print(f'   Image size: {image.shape[1]} x {image.shape[0]} pixels')
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Your Uploaded Image')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file uploaded. Run Option B to use sample images.')

In [ ]:
# ========== OPTION B: USE SAMPLE IMAGES ==========
tumor_number = 1  # Change to 1, 2, or 3

tumor_file = f'tumor_{tumor_number:03d}.png'
print(f'Loading sample: {tumor_file}')

image = load_image_safe(f'data/images/{tumor_file}')
mask = load_mask_safe(f'data/masks/{tumor_file}')

print(f'✅ Loaded tumor {tumor_number}')
print(f'   Image size: {image.shape[1]} x {image.shape[0]} pixels')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title(f'Sample Image {tumor_number}')
axes[0].axis('off')
axes[1].imshow(image, cmap='gray')
axes[1].imshow(mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth (for validation)')
axes[1].axis('off')
plt.show()

---
## 🔬 FFT Analysis

View the frequency spectrum of your image

In [ ]:
# ========== FFT SPECTRUM ANALYSIS ==========
if image is not None:
    print('Computing FFT spectrum...')
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
    
    print('\n📊 FFT Analysis Complete!')
    print('   • Low frequencies (center) = overall shape and contrast')
    print('   • High frequencies (edges) = fine details and boundaries')
else:
    print('⚠️ No image loaded! Run Option A or B above first.')

---
## 🎯 AUTOMATIC TUMOR DETECTION

Run the cell below to automatically detect tumors in your image!

In [ ]:
# ========== AUTOMATIC TUMOR DETECTION ==========
if image is not None:
    print('🔍 Analyzing image for tumors...')
    print('   Using frequency-domain analysis...\n')
    
    # Run detection with combined method (uses both spatial and FFT)
    detection_result = detect_tumor(image, method='combined', sensitivity=0.5)
    
    # Display results
    display_detection_results(image, detection_result)
    
    # If we have ground truth, compare
    if mask is not None:
        print('\n📋 Validation against ground truth:')
        # Calculate accuracy metrics
        detected = detection_result['tumor_mask']
        intersection = np.logical_and(detected, mask).sum()
        union = np.logical_or(detected, mask).sum()
        dice = 2 * intersection / (detected.sum() + mask.sum() + 1e-8)
        iou = intersection / (union + 1e-8)
        
        print(f'   Dice Score: {dice:.3f}')
        print(f'   IoU Score: {iou:.3f}')
        
        # Show comparison
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(image, cmap='gray')
        axes[0].imshow(mask, cmap='Greens', alpha=0.5)
        axes[0].set_title('Ground Truth', fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(image, cmap='gray')
        axes[1].imshow(detected, cmap='Reds', alpha=0.5)
        axes[1].set_title('Our Detection', fontweight='bold')
        axes[1].axis('off')
        
        # Overlap visualization
        overlap = np.zeros((*image.shape, 3))
        overlap[mask > 0] = [0, 1, 0]  # Green = ground truth
        overlap[detected > 0] = [1, 0, 0]  # Red = detected
        overlap[np.logical_and(mask, detected)] = [1, 1, 0]  # Yellow = overlap
        
        axes[2].imshow(image, cmap='gray')
        axes[2].imshow(overlap, alpha=0.5)
        axes[2].set_title('Overlap (Yellow=Correct)', fontweight='bold')
        axes[2].axis('off')
        plt.show()
else:
    print('⚠️ No image loaded! Run Option A or B above first.')

---
## 🔧 Advanced: Compare Detection Methods

See how different methods perform

In [ ]:
# ========== COMPARE ALL METHODS ==========
if image is not None:
    print('Comparing detection methods...\n')
    
    methods = ['otsu', 'fft_highpass', 'fft_bandpass', 'combined']
    results = {}
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.ravel()
    
    for idx, method in enumerate(methods):
        result = detect_tumor(image, method=method, sensitivity=0.5)
        results[method] = result
        
        axes[idx].imshow(image, cmap='gray')
        if result['tumor_detected']:
            axes[idx].imshow(result['tumor_mask'], cmap='Reds', alpha=0.6)
            status = f"DETECTED ({result['confidence']*100:.0f}%)"
        else:
            status = "Not detected"
        
        axes[idx].set_title(f'{method.upper()}\n{status}', fontsize=12, fontweight='bold')
        axes[idx].axis('off')
    
    plt.suptitle('Detection Method Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print('\n' + '='*60)
    print(f'{"Method":<15} {"Detected":<12} {"Confidence":<12} {"Area %":<10}')
    print('='*60)
    for method, result in results.items():
        detected = '✓ Yes' if result['tumor_detected'] else '✗ No'
        conf = f"{result['confidence']*100:.1f}%"
        area = f"{result['tumor_area_percent']:.2f}%"
        print(f'{method:<15} {detected:<12} {conf:<12} {area:<10}')
    print('='*60)
else:
    print('⚠️ No image loaded!')

---
## 📈 Adjust Detection Sensitivity

Try different sensitivity levels to see how it affects detection

In [ ]:
# ========== SENSITIVITY ANALYSIS ==========
if image is not None:
    print('Testing different sensitivity levels...\n')
    
    sensitivities = [0.2, 0.4, 0.6, 0.8]
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    for idx, sens in enumerate(sensitivities):
        result = detect_tumor(image, method='combined', sensitivity=sens)
        
        axes[idx].imshow(image, cmap='gray')
        if result['tumor_detected']:
            axes[idx].imshow(result['tumor_mask'], cmap='Reds', alpha=0.6)
        
        status = f"Area: {result['tumor_area_percent']:.1f}%"
        axes[idx].set_title(f'Sensitivity: {sens}\n{status}', fontsize=11)
        axes[idx].axis('off')
    
    plt.suptitle('Effect of Detection Sensitivity', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print('💡 Higher sensitivity = detects smaller/subtler tumors but may have more false positives')
else:
    print('⚠️ No image loaded!')